# Example notebook

In [1]:
import numpy as np
from anndata import AnnData
import pandas as pd
import compass

In [2]:
adata = AnnData(np.random.normal(size=(20, 10)))

In [4]:
pd.DataFrame().assign(A=["a", "b", "c"], B=[1, 2, 3])

,A,B
0,a,1
1,b,2
2,c,3
